In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# Carga en memoria el dataframe
df = pd.read_pickle('affective_dataset_v2.pkl') 

In [3]:
df.head()
#print(df.columns)

,image,face_closeup,face_landmarks,label
0,"[[165, 166, 164, 167, 167, 166, 167, 167, 167,...","[[161, 160, 161, 161, 161, 161, 161, 160, 99, ...","[[62, 83], [62, 88], [63, 93], [63, 97], [64, ...",bored
1,"[[165, 166, 165, 166, 167, 167, 167, 167, 167,...","[[160, 161, 160, 161, 161, 162, 161, 162, 132,...","[[62, 83], [62, 88], [63, 93], [63, 97], [64, ...",bored
2,"[[165, 165, 165, 166, 167, 166, 166, 167, 167,...","[[160, 161, 161, 60, 36, 76, 88, 88, 92, 102, ...","[[63, 82], [63, 87], [63, 91], [63, 96], [65, ...",bored
3,"[[165, 166, 165, 165, 166, 167, 167, 167, 167,...","[[160, 161, 160, 45, 42, 81, 88, 88, 92, 101, ...","[[63, 83], [63, 87], [63, 92], [64, 97], [65, ...",bored
4,"[[165, 166, 166, 166, 166, 167, 166, 166, 167,...","[[160, 160, 160, 41, 43, 81, 88, 88, 92, 102, ...","[[63, 83], [63, 88], [63, 92], [64, 97], [65, ...",bored


In [4]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [6]:
x = np.array(df['face_landmarks'].tolist())

In [7]:
print(x.shape)

(5023, 144, 2)


In [9]:
print("Original labels:", df['label'].unique())
print("Encoded labels:", label_encoder.classes_)

Original labels: ['bored' 'engaged' 'excited' 'focused' 'interested' 'relaxed']
Encoded labels: ['bored' 'engaged' 'excited' 'focused' 'interested' 'relaxed']


In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [11]:
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

# Entrada del modelo
inputs = Input(shape=(144, 2))

# Aplanar los datos de entrada
flatten = Flatten()(inputs)

# Capas ocultas
dense1 = Dense(600, activation='relu')(flatten)
dense2 = Dense(480, activation='relu')(dense1)
dense3 = Dense(64, activation='relu')(dense2)


# Capa de salida
outputs = Dense(len(label_encoder.classes_), activation='softmax')(dense3)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.65:
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True
            
back = myCallback() 

In [13]:
model.fit(X_train, y_train, epochs=800, batch_size=32, validation_split=0.2, callbacks = [back])

Epoch 1/800
88/88 [==============================] - 3s 10ms/step - loss: 8.2896 - accuracy: 0.3094 - val_loss: 1.7643 - val_accuracy: 0.2685
Epoch 2/800
88/88 [==============================] - 0s 5ms/step - loss: 1.7701 - accuracy: 0.3506 - val_loss: 1.5267 - val_accuracy: 0.4332
Epoch 3/800
88/88 [==============================] - 0s 6ms/step - loss: 1.9693 - accuracy: 0.3343 - val_loss: 1.2911 - val_accuracy: 0.4347
Epoch 4/800
88/88 [==============================] - 1s 8ms/step - loss: 1.4564 - accuracy: 0.3755 - val_loss: 1.3818 - val_accuracy: 0.3949
Epoch 5/800
88/88 [==============================] - 0s 5ms/step - loss: 1.4038 - accuracy: 0.3898 - val_loss: 1.3962 - val_accuracy: 0.3963
Epoch 6/800
88/88 [==============================] - 0s 5ms/step - loss: 1.3874 - accuracy: 0.3745 - val_loss: 1.2794 - val_accuracy: 0.4077
Epoch 7/800
88/88 [==============================] - 0s 5ms/step - loss: 1.3227 - accuracy: 0.4065 - val_loss: 1.2658 - val_accuracy: 0.3949
Epoch 8/800


In [14]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

48/48 [==============================] - 0s 6ms/step - loss: 1.2029 - accuracy: 0.4751
Loss: 1.2029025554656982, Accuracy: 0.47511613368988037


In [15]:
model.save('affective_trained_mlp_475acc.h5')